# 0. Imports, libraries and rusable functions

In [2]:
from project_imports import *
import use_gpu
# Clear any cached memory to start fresh for each trial
torch.cuda.empty_cache()
gc.collect()

Project libraries imported!
GPU: NVIDIA GeForce RTX 4070 Ti SUPER is available.
Device:cuda


76

# 1. Global Variables

In [3]:
## Arguments and global vriables
dataset_name="MCQA-Combined-3"
global_run_name="Optuna-1"
pretrained_model_name = "microsoft/deberta-v3-base"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
data_collator = DefaultDataCollator()
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = right_padding = tokenizer.padding_side == 'right'
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)

c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


# 2. Prepare the Dataset 

In [4]:
# Load the combined dataset
combined_dataset = load_from_disk('cleaned_dataset')

combined_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 1072514
    })
    validation: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 118521
    })
    test: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 200566
    })
})

In [5]:
# Initialize lists to hold datasets
train_datasets = []
val_datasets = []
test_datasets = []

# List of datasets to combine
datasets_to_combine = ['AR-LSAT', 'ReClor', 'LogiQA 2.0']

# Loop through each dataset and filter
for ds_name in datasets_to_combine:
    train_datasets.append(combined_dataset['train'].filter(lambda x: x['Source Dataset'] == ds_name))
    val_datasets.append(combined_dataset['validation'].filter(lambda x: x['Source Dataset'] == ds_name))
    test_datasets.append(combined_dataset['test'].filter(lambda x: x['Source Dataset'] == ds_name))

# Concatenate datasets
combined_train = concatenate_datasets(train_datasets)
combined_val = concatenate_datasets(val_datasets)
combined_test = concatenate_datasets(test_datasets)


# Shuffle the combined dataset
# To ensure that each training batch has a chance to contain a mix of examples from all sources. 
# This helps in reducing variance and improving the generalization of the model.
combined_train = combined_train.shuffle(seed=42)

In [6]:
def mcqa_preprocess_function(examples):
    # Determine the maximum number of choices
    max_num_choices = 5  # Since AR-LSAT has 5 options, we'll pad others to 5
    contexts = examples['Context']
    questions = examples['Question']
    options_list = examples['Options']
    labels = examples['Label']
    
    first_sentences = []
    second_sentences = []
    labels_adjusted = []
    
    for context, question, options, label in zip(contexts, questions, options_list, labels):
        num_choices = len(options)
        # Pad options to have max_num_choices
        if num_choices < max_num_choices:
            options += [''] * (max_num_choices - num_choices)
        first_sentences.append([context] * max_num_choices)
        second_sentences.append([f"{question} {option}" for option in options])
        labels_adjusted.append(label)
    
    # Flatten the lists
    first_sentences = [item for sublist in first_sentences for item in sublist]
    second_sentences = [item for sublist in second_sentences for item in sublist]
    
    # Tokenize the inputs
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=512,
        padding='max_length',
    )
    
    # Un-flatten to shape (num_examples, max_num_choices, seq_length)
    tokenized_inputs = {
        k: [v[i:i + max_num_choices] for i in range(0, len(v), max_num_choices)]
        for k, v in tokenized_examples.items()
    }
    
    # Labels
    tokenized_inputs["labels"] = labels_adjusted
    
    return tokenized_inputs




In [7]:
# Apply the preprocessing function to the combined datasets
encoded_train = combined_train.map(mcqa_preprocess_function, batched=True)
encoded_val = combined_val.map(mcqa_preprocess_function, batched=True)
encoded_test = combined_test.map(mcqa_preprocess_function, batched=True)


In [8]:
# Set the format of the datasets to PyTorch tensors
encoded_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


def get_train_encoded():
    return encoded_train

def get_val_encoded():
    return encoded_val

def get_test_encoded():
    return encoded_test

print("Number of training examples:", len(encoded_train))
print("Number of validation examples:", len(encoded_val))
print("Number of test examples:", len(encoded_test))

Number of training examples: 18290
Number of validation examples: 2299
Number of test examples: 2301


# 3. Reusable Functions

In [8]:
# Load the accuracy metric
accuracy = evaluate.load('accuracy')

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1 = f1_score(labels, predictions, average='weighted')
    return {'eval_accuracy': acc, 'eval_f1': f1}

In [9]:
from transformers import TrainingArguments

def create_training_args(run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=3):
    """
    Generates training arguments for training a machine learning model.

    Parameters:
    - dataset_name (str): The name of the dataset.
    - run_name (str): The name of the run, useful for logging and saving models.
    - model_name (str): The name of the model, typically including its configuration.
    - num_train_epochs (int): The number of epochs to train for.
    - learning_rate (float): The learning rate for training.
    - batch_size (int): The batch size used for training.

    Returns:
    - TrainingArguments: A configured TrainingArguments instance.
    """    
    output_dir = f"./{dataset_name}/{run_name}/{normalized_model_name}"
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",  # Disable all integrations
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=4,
        warmup_steps=398,
        weight_decay=0.194,
        adam_beta1=0.837,
        adam_beta2=0.997,
        adam_epsilon=5.87e-07,
        lr_scheduler_type='cosine',
        fp16=True,  # Enable mixed-precision training
    )
    
    return training_args


In [10]:
def model_init(model_name=pretrained_model_name):
    return AutoModelForMultipleChoice.from_pretrained(model_name)

In [11]:
def create_trainer(model_name=pretrained_model_name,run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=4):
    trainer = Trainer(
        model=model_init(model_name),
        args=create_training_args(run_name=run_name, num_train_epochs=num_train_epochs, learning_rate=learning_rate, batch_size=batch_size),
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    return trainer


In [12]:
class AdvancedEarlyStoppingCallback(TrainerCallback):
    """
    A callback to stop training when either the performance falls below a certain threshold
    or if there is no improvement over a set number of epochs.
    """
    def __init__(self, metric_name, patience):
        self.metric_name = metric_name
        self.patience = patience        
        self.best_score = None
        self.no_improve_epochs = 0
        self.config_file = "early_stopping_config.json"  # Config file for early stopping values

    def read_early_stopping_config(self):
        """
        Reads the early stopping configuration from the file system.
        Returns the configuration as a dictionary.
        """
        if os.path.exists(self.config_file):
            with open(self.config_file, 'r') as file:
                config = json.load(file)
            return config
        else:
            raise FileNotFoundError(f"Config file not found: {self.config_file}")
    def reset_manual_stop_flag(self):
        """
        Resets the manual stop flag to False in the early stopping config file.
        """
        config = self.read_early_stopping_config()
        config['manual_stop'] = False
        with open(self.config_file, 'w') as file:
            json.dump(config, file, indent=4)

    def on_evaluate(self, args, state, control, **kwargs):
        metric_value = kwargs['metrics'].get(self.metric_name)

        if self.best_score is None or metric_value > self.best_score:
            self.best_score = metric_value
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1

        # Check if no improvement has been seen over the allowed patience
        if self.no_improve_epochs >= self.patience:
            control.should_training_stop = True
            print(f"Stopping training: No improvement in {self.metric_name} for {self.patience} epochs")


        # Read the early stopping configuration
        config = self.read_early_stopping_config()
        min_accuracy = config.get("min_accuracy", 0.35)                
        num_epochs_min_acc = config.get("num_epochs_min_acc", 2)  
        max_variance = config.get("max_variance", 0.2)  

        # Check if performance is below the threshold
        if metric_value < min_accuracy:
            control.should_training_stop = True
            print(f"Stopping training: {self.metric_name} below manual min_acc of {min_accuracy}")

         # Manual stop from config
        if config.get("manual_stop", False):
            control.should_training_stop = True
            print(f"Manual early stopping triggered!!")
            self.reset_manual_stop_flag()  # Reset the flag for future runs
            


In [13]:
# Optuna objective function for hyperparameter tuning
def objective(trial):
    # Clear any cached memory to start fresh for each trial
    torch.cuda.empty_cache()
    gc.collect()

    
    model_name = trial.suggest_categorical('model_name', [pretrained_model_name, "./squad-trained-model", './MCQA-Combined/Optuna/trial_5/checkpoint-22865', './MCQA-Combined/Optuna/trial_0/checkpoint-30485','./MCQA-Combined/Optuna/trial_6/checkpoint-30485' ]) 
    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [3, 4])
    #warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    warmup_ratio= trial.suggest_float('warmup_ratio', 0.0, 1.0)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.25)
    adam_beta1 = trial.suggest_float('adam_beta1', 0.8, 0.95)
    adam_beta2 = trial.suggest_float('adam_beta2', 0.990, 0.999)
    adam_epsilon = trial.suggest_float('adam_epsilon', 1e-8, 1e-6)
    lr_scheduler_type = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine', 'cosine_with_restarts']) #,'constant_with_warmup'   
    

    output_dir = f"./{dataset_name}/{global_run_name}/trial_{trial.number}"

    training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",  # Disable all integrations
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=30,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,  # Enable mixed-precision training
    ) 
    
    # Print trial parameters
    print(f"Current Trial {trial.number} parameters: {trial.params}")
    
    trainer = Trainer(
        model=model_init(model_name),
        args=training_args,
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_accuracy', patience=1)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
        
    torch.cuda.empty_cache()  # Clear cache after evaluation
    gc.collect()  # Collect garbage

    return eval_results['eval_accuracy']


# 4. Fine-tuning DeBERTa on the Dataset

## 4.1 Evaluate Vanilla DeBERTa (Acc=19.30%)

In [ ]:
# Create the Trainer
trainer = create_trainer()

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/576 [00:00<?, ?it/s]

## 4.2 Evaluate Trained Vanilla DeBERTa (Acc=27.92%)

In [27]:
# Create the Trainer
trainer = create_trainer()
# Train the model
trainer.train()
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.464600,1.456540,0.256198,0.226952
2,1.468400,1.446175,0.253589,0.234731
3,1.418800,1.430362,0.300565,0.301469


Test Results:
Accuracy: 0.2786
F1 Score: 0.2792


In [13]:
# Create the Trainer
trainer = create_trainer(run_name="Default-Run2", num_train_epochs=10, batch_size=3)
# Train the model
trainer.train()
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as:

  0%|          | 0/60970 [00:00<?, ?it/s]

{'loss': 1.4817, 'grad_norm': 10.460925102233887, 'learning_rate': 4.919968223047422e-05, 'epoch': 0.08}
{'loss': 1.5081, 'grad_norm': 3.530348062515259, 'learning_rate': 4.918816880581236e-05, 'epoch': 0.16}
{'loss': 1.4864, 'grad_norm': 6.0712995529174805, 'learning_rate': 4.916019321314551e-05, 'epoch': 0.25}
{'loss': 1.5478, 'grad_norm': 4.789742469787598, 'learning_rate': 4.911576767182646e-05, 'epoch': 0.33}
{'loss': 1.4785, 'grad_norm': 26.84292984008789, 'learning_rate': 4.905478354129259e-05, 'epoch': 0.41}
{'loss': 1.5623, 'grad_norm': 3.22577166557312, 'learning_rate': 4.897752559048373e-05, 'epoch': 0.49}
{'loss': 1.6182, 'grad_norm': 2.662576198577881, 'learning_rate': 4.8883736139178334e-05, 'epoch': 0.57}
{'loss': 1.6135, 'grad_norm': 2.9794094562530518, 'learning_rate': 4.877361665371831e-05, 'epoch': 0.66}
{'loss': 1.6136, 'grad_norm': 1.978592872619629, 'learning_rate': 4.864724118592782e-05, 'epoch': 0.74}
{'loss': 1.6108, 'grad_norm': 2.1126723289489746, 'learning_r

  0%|          | 0/575 [00:00<?, ?it/s]

{'eval_accuracy': 0.2122662026968247, 'eval_f1': 0.2045627686513225, 'eval_loss': 1.609375, 'eval_runtime': 82.643, 'eval_samples_per_second': 27.818, 'eval_steps_per_second': 6.958, 'epoch': 1.0}


KeyboardInterrupt: 

## 4.3 Optuna Hyperparameters Tuning 1

In [14]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
study.enqueue_trial({'model_name':"microsoft/deberta-v3-base", 'learning_rate': 9.891138752479374e-06, 'batch_size': 3, 'warmup_ratio': 0.5982282303832456, 'weight_decay': 0.17633588993115804, 'adam_beta1': 0.8747290421857349, 'adam_beta2': 0.9927786970263835, 'adam_epsilon': 9.90768817706196e-07, 'lr_scheduler_type': 'linear'})
study.optimize(objective, n_trials=20)


[I 2024-10-16 15:03:56,781] A new study created in memory with name: no-name-0ded4bce-c54e-4ab6-8334-5b865ddd797e


Current Trial 0 parameters: {'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 9.891138752479374e-06, 'batch_size': 3, 'warmup_ratio': 0.5982282303832456, 'weight_decay': 0.17633588993115804, 'adam_beta1': 0.8747290421857349, 'adam_beta2': 0.9927786970263835, 'adam_epsilon': 9.90768817706196e-07, 'lr_scheduler_type': 'linear'}


Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as:

  0%|          | 0/30485 [00:00<?, ?it/s]

{'loss': 1.6108, 'grad_norm': 1.8310006856918335, 'learning_rate': 2.711832744552112e-07, 'epoch': 0.08}
{'loss': 1.6049, 'grad_norm': 2.0155208110809326, 'learning_rate': 5.423665489104224e-07, 'epoch': 0.16}
{'loss': 1.5479, 'grad_norm': 6.674936771392822, 'learning_rate': 8.124650902678128e-07, 'epoch': 0.25}
{'loss': 1.4779, 'grad_norm': 11.677717208862305, 'learning_rate': 1.0825636316252032e-06, 'epoch': 0.33}
{'loss': 1.4543, 'grad_norm': 4.926329612731934, 'learning_rate': 1.3537469060804143e-06, 'epoch': 0.41}


...

{'eval_accuracy': 0.4810787298825576, 'eval_f1': 0.480769738520469, 'eval_loss': 1.1494512557983398, 'eval_runtime': 108.5904, 'eval_samples_per_second': 21.171, 'eval_steps_per_second': 5.295, 'epoch': 2.0}
{'loss': 1.1635, 'grad_norm': 12.913232803344727, 'learning_rate': 1.9346620333868667e-06, 'epoch': 2.08}
{'loss': 1.1512, 'grad_norm': 24.09821128845215, 'learning_rate': 1.841112320828192e-06, 'epoch': 2.19}
{'loss': 1.1273, 'grad_norm': 42.238067626953125, 'learning_rate': 1.7446269228696428e-06, 'epoch': 2.3}
{'loss': 1.1456, 'grad_norm': 35.595550537109375, 'learning_rate': 1.6459364270408495e-06, 'epoch': 2.41}
{'loss': 1.1086, 'grad_norm': 27.671863555908203, 'learning_rate': 1.5455995375745695e-06, 'epoch': 2.51}
{'loss': 1.0868, 'grad_norm': 28.301483154296875, 'learning_rate': 1.4441842792271468e-06, 'epoch': 2.62}
{'loss': 1.1415, 'grad_norm': 31.132102966308594, 'learning_rate': 1.342468740463127e-06, 'epoch': 2.73}
{'loss': 1.1231, 'grad_norm': 64.490234375, 'learning_

  0%|          | 0/575 [00:00<?, ?it/s]

{'eval_accuracy': 0.5176163549369291, 'eval_f1': 0.518378985518002, 'eval_loss': 1.1023606061935425, 'eval_runtime': 130.0531, 'eval_samples_per_second': 17.677, 'eval_steps_per_second': 4.421, 'epoch': 3.0}


[W 2024-10-17 14:46:12,377] Trial 7 failed with parameters: {'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 2.7102291331781023e-06, 'batch_size': 4, 'warmup_ratio': 0.0871495018092514, 'weight_decay': 0.08376945393642041, 'adam_beta1': 0.8976771551751836, 'adam_beta2': 0.9922976863312725, 'adam_epsilon': 6.461136389064594e-07, 'lr_scheduler_type': 'cosine'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\OEM\AppData\Local\Temp\ipykernel_27760\1569860036.py", line 53, in objective
    trainer.train()
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 1938, in train
    return inner_training_loop(
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 2279, in _inner_training_l

KeyboardInterrupt: 

## 4.3 Optuna Hyperparameters Tuning 2

In [14]:
# Create a study object and optimize the objective
study = optuna.create_study(direction='maximize')
#study.enqueue_trial({'model_name': './squad-trained-model', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 3, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})
study.enqueue_trial({'model_name': './MCQA-Combined/Optuna/trial_5/checkpoint-22865', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})
study.enqueue_trial({'model_name': './MCQA-Combined/Optuna/trial_0/checkpoint-30485', 'learning_rate': 9.891138752479374e-06, 'batch_size': 3, 'warmup_ratio': 0.5982282303832456, 'weight_decay': 0.17633588993115804, 'adam_beta1': 0.8747290421857349, 'adam_beta2': 0.9927786970263835, 'adam_epsilon': 9.90768817706196e-07, 'lr_scheduler_type': 'linear'})
study.enqueue_trial({'model_name': './MCQA-Combined/Optuna/trial_6/checkpoint-30485', 'learning_rate': 1.0052258285035737e-05, 'batch_size': 3, 'warmup_ratio': 0.04149176551014211, 'weight_decay': 0.006685281279171756, 'adam_beta1': 0.9429922176765829, 'adam_beta2': 0.9918592948813898, 'adam_epsilon': 8.867767549079712e-08, 'lr_scheduler_type': 'cosine_with_restarts'})
study.enqueue_trial({'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})

study.optimize(objective, n_trials=20)


[I 2024-10-18 01:48:33,486] A new study created in memory with name: no-name-48d19931-bfa9-4c3d-b317-7013bfdd5859


Current Trial 0 parameters: {'model_name': './MCQA-Combined/Optuna/trial_5/checkpoint-22865', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/137190 [00:00<?, ?it/s]

{'loss': 0.2705, 'grad_norm': 99.15497589111328, 'learning_rate': 9.514940314205971e-08, 'epoch': 0.11}
{'loss': 0.2782, 'grad_norm': 0.18223179876804352, 'learning_rate': 1.9125991136636245e-07, 'epoch': 0.22}
{'loss': 0.3224, 'grad_norm': 1.258844256401062, 'learning_rate': 2.873704195906652e-07, 'epoch': 0.33}
{'loss': 0.302, 'grad_norm': 67.60765838623047, 'learning_rate': 3.8348092781496795e-07, 'epoch': 0.44}
{'loss': 0.2937, 'grad_norm': 0.0005371381412260234, 'learning_rate': 4.793992150228221e-07, 'epoch': 0.55}
{'loss': 0.3094, 'grad_norm': 0.0005397977656684816, 'learning_rate': 5.755097232471248e-07, 'epoch': 0.66}
{'loss': 0.3291, 'grad_norm': 14.216509819030762, 'learning_rate': 6.716202314714275e-07, 'epoch': 0.77}
{'loss': 0.3382, 'grad_norm': 1.9903104305267334, 'learning_rate': 7.677307396957303e-07, 'epoch': 0.87}
{'loss': 0.3188, 'grad_norm': 0.03658886253833771, 'learning_rate': 8.636490269035844e-07, 'epoch': 0.98}


...

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/137190 [00:00<?, ?it/s]

{'loss': 1.6108, 'grad_norm': 1.8368643522262573, 'learning_rate': 6.971530295929434e-09, 'epoch': 0.11}


[W 2024-10-18 12:28:51,841] Trial 5 failed with parameters: {'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.2781743075162943e-06, 'batch_size': 4, 'warmup_ratio': 0.6682042061326278, 'weight_decay': 0.11855116033570309, 'adam_beta1': 0.8482673079863498, 'adam_beta2': 0.9938674227791334, 'adam_epsilon': 8.072037956756882e-08, 'lr_scheduler_type': 'linear'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\OEM\AppData\Local\Temp\ipykernel_4464\1873140353.py", line 59, in objective
    trainer.train()
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 1938, in train
    return inner_training_loop(
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 2279, in _inner_training_lo

KeyboardInterrupt: 

## 4.4 Optuna Hyperparameters Tuning 3

## 4.4 Evaluation of Best Model

In [32]:
# Create the Trainer
trainer = create_trainer(model_name="./MCQA-Combined/Optuna/trial_5/checkpoint-22865", run_name="Optuna")
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/576 [00:00<?, ?it/s]

Test Results:
Accuracy: 0.6484
F1 Score: 0.6501


## 4.3 Evaluate SQUAD DeBERTa (Acc=21.9%)

In [16]:
# Create the Trainer
trainer = create_trainer(model_name="./squad-trained-model", run_name="Squad-Run")
# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at ./squad-trained-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/576 [00:00<?, ?it/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


Test Results:
Accuracy: 0.2190
F1 Score: 0.2305


## 4.4 Evaluate Trained SQUAD DeBERTa


In [12]:
path = "./squad-trained-model"
model =  (path)
# Create the Trainer
squad_trainer = create_trainer(run_name="Squad-Run2", batch_size=3, num_train_epochs=10)

# Train the model
squad_trainer.train()

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at ./squad-trained-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mza

  0%|          | 0/60970 [00:00<?, ?it/s]

{'loss': 1.5257, 'grad_norm': 7.596715450286865, 'learning_rate': 4.919968223047422e-05, 'epoch': 0.08}
{'loss': 1.5156, 'grad_norm': 9.40099811553955, 'learning_rate': 4.918816880581236e-05, 'epoch': 0.16}
{'loss': 1.4678, 'grad_norm': 3.4478559494018555, 'learning_rate': 4.916012062578594e-05, 'epoch': 0.25}
{'loss': 1.5513, 'grad_norm': 17.781841278076172, 'learning_rate': 4.911566214483556e-05, 'epoch': 0.33}
{'loss': 1.539, 'grad_norm': 2.640469789505005, 'learning_rate': 4.905492193723485e-05, 'epoch': 0.41}
{'loss': 1.5319, 'grad_norm': 4.278200626373291, 'learning_rate': 4.897752559048373e-05, 'epoch': 0.49}
{'loss': 1.6115, 'grad_norm': 4.821766376495361, 'learning_rate': 4.8883736139178334e-05, 'epoch': 0.57}
{'loss': 1.6179, 'grad_norm': 2.340773344039917, 'learning_rate': 4.877361665371831e-05, 'epoch': 0.66}
{'loss': 1.6139, 'grad_norm': 1.792155146598816, 'learning_rate': 4.864724118592782e-05, 'epoch': 0.74}
{'loss': 1.612, 'grad_norm': 1.8655813932418823, 'learning_rate

  0%|          | 0/575 [00:00<?, ?it/s]

{'eval_accuracy': 0.23053501522401043, 'eval_f1': 0.08637932701924746, 'eval_loss': 1.609375, 'eval_runtime': 84.1784, 'eval_samples_per_second': 27.311, 'eval_steps_per_second': 6.831, 'epoch': 1.0}
{'loss': 1.5381, 'grad_norm': 4.530053615570068, 'learning_rate': 4.7981041883623604e-05, 'epoch': 1.07}
{'loss': 1.5394, 'grad_norm': 6.36260461807251, 'learning_rate': 4.7774877735027944e-05, 'epoch': 1.15}
{'loss': 1.5295, 'grad_norm': 6.978030681610107, 'learning_rate': 4.755312922401073e-05, 'epoch': 1.23}
{'loss': 1.5172, 'grad_norm': 4.914771556854248, 'learning_rate': 4.731594546933761e-05, 'epoch': 1.31}
{'loss': 1.5025, 'grad_norm': 5.194087505340576, 'learning_rate': 4.706348596948085e-05, 'epoch': 1.39}
{'loss': 1.5084, 'grad_norm': 6.125194549560547, 'learning_rate': 4.679592049536167e-05, 'epoch': 1.48}
{'loss': 1.5085, 'grad_norm': 7.808128356933594, 'learning_rate': 4.651342897618479e-05, 'epoch': 1.56}
{'loss': 1.5049, 'grad_norm': 4.580046653747559, 'learning_rate': 4.621

...

  0%|          | 0/575 [00:00<?, ?it/s]

{'eval_accuracy': 0.23053501522401043, 'eval_f1': 0.08637932701924746, 'eval_loss': 1.6093724966049194, 'eval_runtime': 83.0172, 'eval_samples_per_second': 27.693, 'eval_steps_per_second': 6.926, 'epoch': 9.0}
{'loss': 1.4614, 'grad_norm': 4.2972731590271, 'learning_rate': 1.1799981822759164e-06, 'epoch': 9.02}
{'loss': 1.4505, 'grad_norm': 4.579303741455078, 'learning_rate': 9.926857147363265e-07, 'epoch': 9.1}
{'loss': 1.4744, 'grad_norm': 2.0948286056518555, 'learning_rate': 8.212484093368643e-07, 'epoch': 9.18}
{'loss': 1.4763, 'grad_norm': 2.1727755069732666, 'learning_rate': 6.658015521700958e-07, 'epoch': 9.27}
{'loss': 1.4798, 'grad_norm': 4.078769683837891, 'learning_rate': 5.264496762647245e-07, 'epoch': 9.35}
{'loss': 1.4521, 'grad_norm': 5.194500923156738, 'learning_rate': 4.0328649129046553e-07, 'epoch': 9.43}
{'loss': 1.4733, 'grad_norm': 4.355737686157227, 'learning_rate': 2.9639482054129713e-07, 'epoch': 9.51}
{'loss': 1.4531, 'grad_norm': 2.2465782165527344, 'learning_

  0%|          | 0/575 [00:00<?, ?it/s]

{'eval_accuracy': 0.23053501522401043, 'eval_f1': 0.08637932701924746, 'eval_loss': 1.609375, 'eval_runtime': 82.9839, 'eval_samples_per_second': 27.704, 'eval_steps_per_second': 6.929, 'epoch': 10.0}
{'train_runtime': 23204.0495, 'train_samples_per_second': 7.882, 'train_steps_per_second': 2.628, 'train_loss': 1.4887918260501452, 'epoch': 10.0}


TrainOutput(global_step=60970, training_loss=1.4887918260501452, metrics={'train_runtime': 23204.0495, 'train_samples_per_second': 7.882, 'train_steps_per_second': 2.628, 'total_flos': 2.40617214893568e+17, 'train_loss': 1.4887918260501452, 'epoch': 10.0})

In [13]:
# Evaluate the model on the test set
test_results = squad_trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

  0%|          | 0/576 [00:00<?, ?it/s]

Test Results:
Accuracy: 0.2190
F1 Score: 0.0898


In [27]:
path = "./MCQA-Combined/Squad-Run/microsoft-deberta-v3-base/checkpoint-18291"
model =  AutoModelForMultipleChoice.from_pretrained(path)
# Create the Trainer
squad_trainer = create_trainer(run_name="Squad-Run2", batch_size=3)

# Evaluate the model on the test set
test_results = squad_trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

C:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mzak071 (COMPSCI714). Use `wandb login --relogin` to force relogin


Test Results:
Accuracy: 0.2873
F1 Score: 0.2872


In [45]:
# Train the model
squad_trainer.train()

# Evaluate the model on the test set
test_results = squad_trainer.evaluate(eval_dataset=get_test_encoded())
print("Test Results:")
print(f"Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"F1 Score: {test_results['eval_f1']:.4f}")

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# End of NoteBook